## Курсова робота на тему "Визначення найбільш вигідної роздрібної ціни продажу товару в магазині з урахуванням плати на кредит"
## Виконав студент гр. ІС-72 Цилюрик Дмитро

### Програмний опис імітації мережі Петрі використовуючи ООП підхід

##### Імпорт необхідних бібліотек

In [1]:
import math
import numpy as np
import pandas as pd
import sys
from enum import Enum
from IPython.display import display

##### Клас збереження типів розподілу для ренерації затримок

In [2]:
class Distribution(Enum):
    EXPONENTIAL = np.random.exponential
    UNIFORM = np.random.uniform
    NORMAL = np.random.normal
    POISSON = np.random.poisson

##### Клас для генерації затримок

In [3]:
class RandGenerator:
    @staticmethod
    def generate(distribution_function, **args):
        return distribution_function(**args)

##### Клас для представлення Позиції в мережі Петрі

In [4]:
class Position:
    def __init__(self, num_of_markers=0, description=None, **kwargs):
        self.num_of_markers = num_of_markers
        self.description = description

        self.in_arcs = []
        self.out_arcs = []

    def add_markers(self, num_of_markers):
        self.num_of_markers += num_of_markers

    def remove_markers(self, num_of_markers):
        self.num_of_markers -= num_of_markers

    def __repr__(self):
        return f"Position<{self.description}> = {self.num_of_markers}"

##### Клас для представлення Переходу в мережі Перті

In [5]:
class Transition:
    def __init__(
        self,
        delay=0,
        delay_distribution=None,
        delay_distribution_params=None,
        priority=1,
        probability=None,
        description=None,
        **kwargs,
    ):
        self.delay = delay
        self.delay_distribution = delay_distribution
        self.delay_distribution_params = delay_distribution_params
        self.priority = priority
        self._probability = probability
        self.description = description

        self.in_arcs = []
        self.out_arcs = []

        self.markers_release_times = {}

    @property
    def probability(self):
        return self._probability
    
    def _update_markers_release_times(self, release_timestamp):
        if release_timestamp in self.markers_release_times:
            self.markers_release_times[release_timestamp] += 1
        else:
            self.markers_release_times[release_timestamp] = 1
    
    def get_delay(self):
        return round(
            self.delay 
            if self.delay_distribution is None 
            else RandGenerator.generate(
                self.delay_distribution.value, **self.delay_distribution_params
            )
        )

    def check_can_perform_move_from_transition(self):
        return all(arc.check_move_possibility() for arc in self.in_arcs)

    def perform_move_from_transition(self, current_time):
        delay = self.get_delay()
        release_timestamp = current_time + delay
        self._update_markers_release_times(release_timestamp)
        for arc in self.in_arcs:
            arc.move_from()

    def check_can_perform_move_to_transition(self, current_time):
        return current_time in self.markers_release_times.keys()
    
    def perform_move_to_transition(self, current_time):
        for _ in range(self.markers_release_times[current_time]):
            for arc in self.out_arcs:
                arc.move_to()
        else:
            del self.markers_release_times[current_time]

    def __repr__(self):
        return f"Transition<{self.description}> = {self.markers_release_times}"

##### Клас для представлення Переходу в мережі Перті із заданою функцієї умовірностей переходу

In [20]:
class TransitionWithProbabilityFunc(Transition):
    def __init__(self, probability_func, **kwargs):
        super().__init__(**kwargs)
        self._queue = kwargs["queue"]
        self._m = kwargs["m"]
        self._probability_func = probability_func

    @property
    def probability(self):
        return self._probability_func(self._queue.num_of_markers, self._m)

##### Клас для представлення Дуги в мережі Перті

In [21]:
class Arc:
    def __init__(self, start, end, multiplier=1, informational=False):
        self.start = start
        self.end = end
        self.multiplier = multiplier
        self.informational = informational

        self._set_arc_ends()

    def _set_arc_ends(self):
        self.start.out_arcs.append(self)
        self.end.in_arcs.append(self)

    def check_move_possibility(self):
        return self.start.num_of_markers >= self.multiplier

    def move_from(self):
        if not self.informational:
            self.start.remove_markers(self.multiplier)

    def move_to(self):
        self.end.add_markers(self.multiplier)

    def __repr__(self):
        return f"{self.start} --> {self.end}"

##### Клас для представлення Дуги в мережі Перті за умови переходу всіх мартерів із Позиції

In [22]:
class ArcAll(Arc):
    def __init__(self, move_to_all=True, **kwargs):
        super().__init__(**kwargs)
        self._move_to_all = move_to_all

    def check_move_possibility(self):
        return self.start.num_of_markers >= 0

    def move_from(self):
        self.end.multiplier_all = self.start.num_of_markers
        if not self.informational:
            self.start.remove_markers(self.start.num_of_markers)

    def move_to(self):
        if self._move_to_all:
            self.end.add_markers(self.start.multiplier_all)
        else:
            super().move_to()

##### Клас для представлення Дуги в мережі Перті за умови переходу всіх мартерів із Позиції за заданої функцію визначення кількості маркерів для переходу

In [23]:
class ArcAllWithMoveToFunc(ArcAll):
    def __init__(self, move_to_func, **kwargs):
        super().__init__(**kwargs)
        self._move_to_func = move_to_func
    
    def move_to(self):
        if self._move_to_all:
            self.end.add_markers(self._move_to_func(self.start.multiplier_all))
        else:
            super().move_to()

##### Клас моделювання мережі Петрі

In [186]:
class Model:
    def __init__(
        self, 
        transitions, 
        positions, 
        modeling_time,
        stop_condition=None,
        verbose=False,
    ):
        self.transitions = transitions
        self.positions = positions
        self._modeling_time = modeling_time
        self._verbose = verbose
        
        if stop_condition is None:
            self._stop_condition = lambda : True
        else:
            self._stop_condition = stop_condition
        
        self.current_time = 0
        self.position_markers_stats = {position: [] for position in positions}

    def run(self):
        if self._verbose:
            self._display_model_status()
        while self.current_time < self._modeling_time:
            while True:       
                if not any(
                    transition.check_can_perform_move_from_transition()
                    for transition in self.transitions
                ):
                    break
                self._move()
                self.update_statistics()
                
            if not any(
                len(transition.markers_release_times) > 0
                for transition in self.transitions
            ) or self._stop_condition():
                break
            self._move()
            self.update_statistics()
            
            self.current_time += 1
            if self._verbose:
                self._display_model_status()

        return self.get_result_statistics()

    def _move(self):
        transitions_without_conflicts = self._get_transitions_with_resolved_conflicts()
        for transition in transitions_without_conflicts:
            if transition.check_can_perform_move_from_transition():
                transition.perform_move_from_transition(self.current_time)
        for transition in self.transitions:
            if transition.check_can_perform_move_to_transition(self.current_time):
                transition.perform_move_to_transition(self.current_time)
  
    def _get_transitions_with_resolved_conflicts(self):
        conflicting_transitions = []
        resulting_transitions = []

        for position in self.positions:
            non_informational_arcs = filter(lambda arc: not arc.informational, position.out_arcs)

            valid_transitions = [*filter(
                lambda transition: transition.check_can_perform_move_from_transition(),
                map(lambda arc: arc.end, non_informational_arcs)
            )]

            if len(valid_transitions) > 1:
                conflicting_transitions.append(valid_transitions)

        for conflict in conflicting_transitions:
            is_probabilistic_conflict = conflict[0].probability is not None

            if is_probabilistic_conflict:
                probabilities = [*map(lambda transition: transition.probability, conflict)]

                gap = 1 - sum(probabilities)
                if gap != 0:

                    probabilities = [p + gap / len(probabilities) for p in probabilities]

                resulting_transitions.append(np.random.choice(conflict, p=probabilities))
            else:
                conflict.sort(key=lambda transition: transition.priority, reverse=True)
                resulting_transitions.append(conflict[0])


        for transition in self.transitions:
            was_transition_in_conflict = any([transition in conflict for conflict in conflicting_transitions])

            if not was_transition_in_conflict:
                resulting_transitions.append(transition)

        return resulting_transitions

    def update_statistics(self):
        for position in self.positions:
            self.position_markers_stats[position].append(position.num_of_markers)

    def get_result_statistics(self):
        result_stats = []

        for position, stats in self.position_markers_stats.items():
            result_stats.append(
                {
                    "description": position.description,
                    "avg_markers": sum(stats) / len(stats),
                    "max_markers": max(stats),
                    "min_markers": min(stats),
                    "result_markers": stats[-1],
                    "time_modeling": self.current_time,
                }            
            )

        return result_stats

    def _display_model_status(self):
        print(f"Current time: {self.current_time}")
        for position in self.positions:
            print(position)
        for transition in self.transitions:
            print(transition)
        print()

![Model](diagrams/petri_model.png "Мережа Петрі для задачі")

### Моделювання задачі

In [201]:
class StoreSystem:
    LAMBDA = 30
    k = 0.02
    s = 500
    N = 1000

    p1 = {
        (0, 5): 0.25,
        (6, 10): 0.25,
        (11, 20): 0.2,
        (21, 50): 0.2,
        (51, sys.maxsize): 0.1,
    }
    p2 = {
        (0, 1.4999): 0.3,
        (1.5, 1.9999): 0.3,
        (2.0, 2.9999): 0.2,
        (3.0, 3.9999): 0.1,
        (4.0, 5.9999): 0.06,
        (6.0, 9.9999): 0.04,
        (10.0, sys.maxsize): 0,
    }

    A = 30
    B = 5

    M = 1000
    minutes_in_one_day = 24 * 60
    modeling_time = math.inf * minutes_in_one_day
       
    def __init__(self, m):
        self.m = m
        
        # Positions
        self.customer_generator = Position(num_of_markers=1, description="Потік покупців")
        self.customer_came = Position(num_of_markers=0, description="Покупець надійшов")
        self.queue = Position(num_of_markers=0, description="Черга покупців")
        self.customer_left_store = Position(num_of_markers=0, description="Кількість покупців, що покинули мазагин")
        self.stock = Position(num_of_markers=self.N, description="Запас товару")
        self.seller = Position(num_of_markers=1, description="Продавець")
        self.customer_success = Position(num_of_markers=0, description="Кількість обслугованих покупців")
        self.profit = Position(num_of_markers=0, description="Прибуток")
        self.debt = Position(num_of_markers=self.N * self.s * self.M, description="Борг")
        self.repaid_debt = Position(num_of_markers=0, description="Погашений борг")
        self.delay_generator = Position(num_of_markers=1, description="Затримка процентної ставки")
        self.interest_rate_ready = Position(num_of_markers=0, description="Процентна ставка готова до дії")
        
        # Transitions
        self.customer_arrive = Transition(
            delay_distribution=Distribution.EXPONENTIAL,
            delay_distribution_params={"scale": self.LAMBDA},
            description="Надходження покупця"
        )
        self.customer_leave_store = TransitionWithProbabilityFunc(
            probability_func=self.get_leave_probability, 
            queue=self.queue,
            m=self.m,
            description="Покупець покипає магазин"
        )
        self.customer_stay_in_store = TransitionWithProbabilityFunc(
            probability_func=self.get_stay_in_store_probability, 
            queue=self.queue,
            m=self.m,
            description="Покупець стає в чергу"
        )
        self.service = Transition(
            delay_distribution=Distribution.UNIFORM,
            delay_distribution_params={"low": self.A - self.B, "high": self.A + self.B},
            description="Обслуговування покупця"
        )
        self.debt_repayment = Transition(description="Гасіння боргу")
        self.interest_rate_delay = Transition(delay=self.minutes_in_one_day, description="Затримка для процентнох ставки")
        self.interest_rate_effect = Transition(description="Дія процентної ставки", priority=2)
        
        # Arcs
        Arc(start=self.customer_generator, end=self.customer_arrive)
        Arc(start=self.customer_arrive, end=self.customer_generator)
        Arc(start=self.customer_arrive, end=self.customer_came)
        Arc(start=self.customer_came, end=self.customer_leave_store)
        Arc(start=self.customer_leave_store, end=self.customer_left_store)
        Arc(start=self.customer_came, end=self.customer_stay_in_store)
        Arc(start=self.customer_stay_in_store, end=self.queue)
        Arc(start=self.queue, end=self.service)
        Arc(start=self.stock, end=self.service)
        Arc(start=self.seller, end=self.service)
        Arc(start=self.service, end=self.seller)
        Arc(start=self.service, end=self.customer_success)
        Arc(start=self.service, end=self.profit, multiplier=int(self.m * self.s * self.M))
        Arc(start=self.profit, end=self.debt_repayment, multiplier=self.M)
        Arc(start=self.debt, end=self.debt_repayment, multiplier=self.M)
        Arc(start=self.debt_repayment, end=self.repaid_debt, multiplier=self.M)
        Arc(start=self.delay_generator, end=self.interest_rate_delay)
        Arc(start=self.interest_rate_delay, end=self.delay_generator)
        Arc(start=self.interest_rate_delay, end=self.interest_rate_ready)
        Arc(start=self.interest_rate_ready, end=self.interest_rate_effect)
        ArcAll(start=self.debt, end=self.interest_rate_effect)
        ArcAllWithMoveToFunc(start=self.interest_rate_effect, end=self.debt, move_to_func=self.calculate_debt_rate_effect)


    @staticmethod
    def get_stay_in_store_probability(num_in_queue, m):
        p1_current = next(filter(lambda item: item[0][0] <= num_in_queue <= item[0][1], StoreSystem.p1.items()))[1]
        p2_current = next(filter(lambda item: item[0][0] <= m <= item[0][1], StoreSystem.p2.items()))[1]
        return p1_current * p2_current

    @staticmethod
    def get_leave_probability(num_in_queue, m):
        return 1 - StoreSystem.get_stay_in_store_probability(num_in_queue, m)

    @staticmethod
    def calculate_debt_rate_effect(current_debt):
        return round(current_debt * (1 + StoreSystem.k))
    
    def _stop_condition(self):
        return (
            True 
            if self.stock.num_of_markers == 0 and len(self.service.markers_release_times) == 0
            else False
        )
    
    def _append_target_function_result(self, modeling_results):
        modeling_results.append(
            {
                "description": "Чистий прибуток",
                "avg_markers": None,
                "max_markers": None,
                "min_markers": None,
                "result_markers": (self.profit.num_of_markers - self.debt.num_of_markers) / StoreSystem.M,
                "time_modeling": modeling_results[0]["time_modeling"],
            }
        )
        return modeling_results

    def simulate(self, verbose=False, flag_display_results=True):
        net = Model(
            positions=[
                self.customer_generator,
                self.customer_came,
                self.queue,
                self.customer_left_store,
                self.stock,
                self.seller,
                self.customer_success,
                self.profit,
                self.debt,
                self.repaid_debt,
                self.delay_generator,
                self.interest_rate_ready,
            ],
            transitions=[
                self.customer_arrive,
                self.customer_leave_store,
                self.customer_stay_in_store,
                self.service,
                self.debt_repayment,
                self.interest_rate_delay,
                self.interest_rate_effect,
            ],
            modeling_time=self.modeling_time,
            stop_condition=self._stop_condition,
            verbose=verbose,
        )
        modeling_results = self._append_target_function_result(net.run())
        
        if flag_display_results:
            display(pd.DataFrame(modeling_results))
        
        return modeling_results


### Оцінка адекватності моделі

In [202]:
_system = StoreSystem(1.7)
test_simulation = _system.simulate()

,description,avg_markers,max_markers,min_markers,result_markers,time_modeling
0,Потік покупців,1.037946e-02,1.000000e+00,0.0,0.000000e+00,411539
1,Покупець надійшов,1.037946e-02,1.000000e+00,0.0,0.000000e+00,411539
2,Черга покупців,2.298013e-03,1.000000e+00,0.0,1.000000e+00,411539
3,"Кількість покупців, що покинули мазагин",6.181312e+03,1.224200e+04,0.0,1.224200e+04,411539
4,Запас товару,5.019014e+02,1.000000e+03,0.0,0.000000e+00,411539
5,Продавець,9.249721e-01,1.000000e+00,0.0,1.000000e+00,411539
6,Кількість обслугованих покупців,4.980236e+02,1.000000e+03,0.0,1.000000e+03,411539
7,Прибуток,2.834712e+05,8.500000e+05,0.0,0.000000e+00,411539
8,Борг,1.760730e+10,9.788040e+10,493200000.0,9.787870e+10,411539
9,Погашений борг,4.230366e+08,8.500000e+08,0.0,8.500000e+08,411539


#### Середній час надходження покупців

In [224]:
num_of_clients = 0
for item in test_simulation:
    if item["description"] in ["Кількість покупців, що покинули мазагин", "Кількість обслугованих покупців"]:
        num_of_clients += item["result_markers"]

print(f"Average arrival time: {test_simulation[0]['time_modeling']/num_of_clients}")

31.07831143331823

Як можна помітити, середній час надходження покупців є близьким до дійсності. За умовою клієнти надходитя в чередньому кожні 30 хлилин (пуасонівський потік із параметром λ = 1/30 ($\frac{1}{хв}$)

#### Ймовірність купівлі товару

In [230]:
failed_num_of_clients, succeeded_num_of_clients = 0, 0
for item in test_simulation:
    if item["description"] == "Кількість покупців, що покинули мазагин":
        failed_num_of_clients += item["result_markers"]
    elif item["description"] == "Кількість обслугованих покупців":
        succeeded_num_of_clients += item["result_markers"]

print(f"Average probability of purchase: {succeeded_num_of_clients/(succeeded_num_of_clients+failed_num_of_clients)}")

Average probability of purchase: 0.07551729346020239


При запуску тестового прогону був використаний коефіцієнт ціни товару: $m = 1.7$. Це відповідає значенню $p_2=0.3$. Середнє значення покупців в черзі рівне 0 (див. середнє значення в позиції "Черга покупців"), тому $p_1=0.25$. Отримаємо наступну ймовірність купівлі товару покупцем: 
\begin{equation*}
P = p_1*p_2 = 0.3*0.25 = 0.075
\end{equation*}
Теоретична та експериментальна частота покупок є майже однаковими. Отже, модель є адекватною.

### Визначення кількості прогонів
Надалі будемо використовувати дану формулу:
\begin{equation*}
N = \frac{t_α^2σ^2}{ε^2}+1
\end{equation*}
де $σ^2$ - дисперсія відгуку моделі, $ε$ - точність вимірювання, $t_α$ - агрумент функції Лаплача

In [203]:
experiments = []
for i in np.arange(20):
    print(f"Experiment {i}")
    m = 1.7
    store_system = StoreSystem(m)
    for result_item in store_system.simulate(flag_display_results=False): 
        experiments.append(
            {
                "price_coefficient":m,
                **result_item
            }
        )

Experiment 0
Experiment 1
Experiment 2
Experiment 3
Experiment 4
Experiment 5
Experiment 6
Experiment 7
Experiment 8
Experiment 9
Experiment 10
Experiment 11
Experiment 12
Experiment 13
Experiment 14
Experiment 15
Experiment 16
Experiment 17
Experiment 18
Experiment 19


In [205]:
experiments_df = pd.DataFrame(experiments)
profit_df = experiments_df.loc[experiments_df["description"] == "Чистий прибуток"]

In [206]:
display(profit_df)

,price_coefficient,description,avg_markers,max_markers,min_markers,result_markers,time_modeling
12,1.7,Чистий прибуток,NaN,NaN,NaN,-8.025148e+07,396926
25,1.7,Чистий прибуток,NaN,NaN,NaN,-9.037609e+07,406900
38,1.7,Чистий прибуток,NaN,NaN,NaN,-1.118512e+08,420524
51,1.7,Чистий прибуток,NaN,NaN,NaN,-9.347771e+07,409732
64,1.7,Чистий прибуток,NaN,NaN,NaN,-1.154950e+08,422022
77,1.7,Чистий прибуток,NaN,NaN,NaN,-1.466659e+08,437640
90,1.7,Чистий прибуток,NaN,NaN,NaN,-8.525043e+07,403171
103,1.7,Чистий прибуток,NaN,NaN,NaN,-1.138227e+08,420556
116,1.7,Чистий прибуток,NaN,NaN,NaN,-1.196249e+08,427261
129,1.7,Чистий прибуток,NaN,NaN,NaN,-7.109059e+07,388018


In [221]:
needed_number_of_experiments = round(
    (1.96**2*profit_df.std()["result_markers"]**2) / 
    (profit_df.mean()["result_markers"]*0.15)**2 
    + 1
)

In [225]:
print(f"Number of needed experiments: {needed_number_of_experiments}")

Number of needed experiments: 11.0


### Аналіз та оцінка результатів

Для визначення впливу факторів на значення цільової функції було використано дисперсійний аналіз. Основне питання, на яке дає відповідь дисперсійний аналіз впливу фактора, формулюється так: різниця у значеннях відгуку моделі, отриманих при різних значеннях фактора обумовлена випадковістю, чи пояснюється виключно дією фактора?
Надалі будуть використні наступні формули:
- для розрахунку середнього значення цільової функції заданого коефіцієнта $j$
\begin{equation*}
\overline{y_j} = \frac{1}{n}\sum _{i=1}^{n}y_{ij}, 
\end{equation*}
де $n$ - кількість прогонів
- для розрахунку середнього значення цільової функції
\begin{equation*}
\overline{y} = \frac{1}{m}\sum _{j=1}^{m}y_{j}, 
\end{equation*}
де $m$ - кількість коефіцієнтів
- для розрахунку значення різниць між групами коефіцієнтів
\begin{equation*}
d_{факт} = n \sum _{j=1}^{m}(\overline{y_{j}}-\overline{y})^2, 
\end{equation*}
де $n$ - кількість прогонів, $m$ - кількість коефіцієнтів
- для розрахунку значення через групами коефіцієнтів
\begin{equation*}
d_{залиш} = \frac{\sum _{i=1}^{K}\sum _{j=1}^{n_i}(y_{ij}-\overline{y_{i}})^2}{N-K}, 
\end{equation*}
\begin{equation*}
N = m*n,
\end{equation*}
\begin{equation*}
K = m
\end{equation*}
де $n$ - кількість прогонів, $m$ - кількість коефіцієнтів
- для розрахунку значення критерію Фішера
\begin{equation*}
F = \frac{d_{факт}}{d_{залиш}}
\end{equation*}

Виходячи з контексту даної задачі є доцільним обрати **коефіцієнт ціни товару** (**_price_coefficient_**) фактором, при цьому відгуком моделі буде значення цільової функції, тобто "**Чистий прибуток**" ("**_Прибуток_**" - "**_Борг_**")

In [236]:
results_first_iteration = []
for m in [1.0, 1.4, 1.9, 2.9, 3.9, 5.9, 9.9]:
    for i in np.arange(needed_number_of_experiments):
        print(f"price_coefficient: {m}; experiment {i}")
        store_system = StoreSystem(m)
        for result_item in store_system.simulate(flag_display_results=False): 
            results_first_iteration.append(
                {
                    "experiment": i,
                    "price_coefficient":m,
                    **result_item
                }
            )

price_coefficient: 1.0; experiment 0.0
price_coefficient: 1.0; experiment 1.0
price_coefficient: 1.0; experiment 2.0
price_coefficient: 1.0; experiment 3.0
price_coefficient: 1.0; experiment 4.0
price_coefficient: 1.0; experiment 5.0
price_coefficient: 1.0; experiment 6.0
price_coefficient: 1.0; experiment 7.0
price_coefficient: 1.0; experiment 8.0
price_coefficient: 1.0; experiment 9.0
price_coefficient: 1.0; experiment 10.0
price_coefficient: 1.4; experiment 0.0
price_coefficient: 1.4; experiment 1.0
price_coefficient: 1.4; experiment 2.0
price_coefficient: 1.4; experiment 3.0
price_coefficient: 1.4; experiment 4.0
price_coefficient: 1.4; experiment 5.0
price_coefficient: 1.4; experiment 6.0
price_coefficient: 1.4; experiment 7.0
price_coefficient: 1.4; experiment 8.0
price_coefficient: 1.4; experiment 9.0
price_coefficient: 1.4; experiment 10.0
price_coefficient: 1.9; experiment 0.0
price_coefficient: 1.9; experiment 1.0
price_coefficient: 1.9; experiment 2.0
price_coefficient: 1.9;

In [248]:
results_first_iteration_df = pd.DataFrame(results_first_iteration)
first_iteration_profit_df = results_first_iteration_df.loc[results_first_iteration_df["description"] == "Чистий прибуток"]

In [258]:
first_iteration_profit_df["result_markers"] = first_iteration_profit_df["result_markers"].astype("float64")

net_revenues = first_iteration_profit_df["result_markers"]
global_average = sum(net_revenues) / len(net_revenues)

avg_net_revenues = first_iteration_profit_df.groupby(["price_coefficient"])["result_markers"].mean()
df_with_avg = first_iteration_profit_df.merge(avg_net_revenues, on=["price_coefficient"], suffixes=('', '_avg'))
df_with_avg["group_deviation"] = (df_with_avg["result_markers"] - df_with_avg["result_markers_avg"])**2
num_of_groups = len(avg_net_revenues)

s_factual = needed_number_of_experiments * sum((avg_net_revenues - global_average)**2)
s_residual = sum(df_with_avg["group_deviation"])

d_factual = s_factual
d_residual = s_residual / ((num_of_groups * needed_number_of_experiments) - num_of_groups)

F = d_factual / d_residual
F_critical = 1.63565309

print(f"Sum of Squares: {d_factual}")
print(f"Degrees of Freedom: {((num_of_groups * needed_number_of_experiments) - num_of_groups)}")
print(f"Mean Square: {d_residual}")
print(f"F: {F}; F-critical(alpha=0.05):{F_critical}")
print(f"Factor is {'not ' if F < F_critical else ''}significant")

Sum of Squares: 2.527396837249382e+49
Degrees of Freedom: 70.0
Mean Square: 3.744922990530315e+47
F: 67.48861975640999; F-critical(alpha=0.05):1.63565309
Factor is significant


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [279]:
first_iteration_profit_df["rank"] = first_iteration_profit_df["result_markers"].rank(method="max", ascending=False)
first_iteration_profit_df.sort_values(by=['rank'], inplace=True)
first_iteration_profit_df["previous_result_markets"] = first_iteration_profit_df['result_markers'].shift(1)
first_iteration_profit_df["previous_current_differents"] = first_iteration_profit_df["result_markers"] - first_iteration_profit_df["previous_result_markets"]

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

In [283]:
pd.set_option('display.float_format', lambda x: '%.1f' % x)

In [284]:
first_iteration_profit_df

,experiment,price_coefficient,description,avg_markers,max_markers,min_markers,result_markers,time_modeling,rank,previous_result_markets,previous_current_differents
155,0.0,1.4,Чистий прибуток,nan,nan,nan,-63652205.7,377098,1.0,nan,nan
194,3.0,1.4,Чистий прибуток,nan,nan,nan,-79959085.5,392584,2.0,-63652205.7,-16306879.8
272,9.0,1.4,Чистий прибуток,nan,nan,nan,-81146643.1,393213,3.0,-79959085.5,-1187557.6
389,7.0,1.9,Чистий прибуток,nan,nan,nan,-81557948.3,402853,4.0,-81146643.1,-411305.2
181,2.0,1.4,Чистий прибуток,nan,nan,nan,-82237751.6,393550,5.0,-81557948.3,-679803.3
324,2.0,1.9,Чистий прибуток,nan,nan,nan,-83518096.7,402038,6.0,-82237751.6,-1280345.1
363,5.0,1.9,Чистий прибуток,nan,nan,nan,-84158341.6,403595,7.0,-83518096.7,-640244.9
428,10.0,1.9,Чистий прибуток,nan,nan,nan,-84575069.6,402041,8.0,-84158341.6,-416728.0
350,4.0,1.9,Чистий прибуток,nan,nan,nan,-86198839.5,407286,9.0,-84575069.6,-1623769.9
168,1.0,1.4,Чистий прибуток,nan,nan,nan,-86699382.4,396536,10.0,-86198839.5,-500542.9


[![Free Statistics Calculators](references/free_statistics_calculators.png "Free Statistics Calculators")](https://www.danielsoper.com/statcalc/calculator.aspx?id=4)

In [243]:
import json
res_from_file = []
with open("/Users/dtsylyuryk/Work/KPI/system-modeling-course-work/experiments/price_coefficient_change_11_runs_1_iteration.json", "r") as f:
    for line in f.readlines():
        res_from_file.append(json.loads(line))


In [237]:
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)
pd.set_option('display.width', 1000)

In [63]:
min_ = np.inf
for item in results:
    if item["description"] == "Богр" and item['result_markers'] < min_:
        min_ = item['result_markers']

In [64]:
min_

74189643203

In [52]:
_system = StoreSystem(1.999)
_system.simulate()


# LAMBDA = 20
# k = 0.01
# s = 1000
# N = 600

# p1 = {
#     (0, 5): 0.3,
#     (6, 10): 0.3,
#     (11, 20): 0.2,
#     (21, 50): 0.1,
#     (51, sys.maxsize): 0.1,
# }
# p2 = {
#     (0, 1.4999): 0.45,
#     (1.5, 1.9999): 0.4,
#     (2.0, 2.9999): 0.06,
#     (3.0, 3.9999): 0.04,
#     (4.0, 5.9999): 0.03,
#     (6.0, 9.9999): 0.02,
#     (10.0, sys.maxsize): 0,
# }

# A = 30
# B = 5

# m = 1.999
# M = 1000
# minutes_in_one_day = 24 * 60
# modeling_period = 90 * minutes_in_one_day


Modeling time spent: 103447


,description,avg_markers,max_markers,min_markers,result_markers
0,Потік покупців,5.801028e-03,1,0,0
1,Покупець надійшов,5.801028e-03,1,0,0
2,Черга покупців,9.675905e-03,2,0,0
3,"Кількість покупців, що покинули мазагин",1.397326e+03,4282,0,4282
4,Запас товару,4.004802e+02,600,0,0
5,Продавець,8.321457e-01,1,0,1
6,Кількість обслугованих покупців,1.993519e+02,600,0,600
7,Прибуток,1.289389e+07,466719000,0,466719000
8,Богр,3.028773e+08,600000000,588,776
9,Погашений борг,3.856106e+08,732681000,0,732681000


[{'description': 'Потік покупців',
  'avg_markers': 0.005801027834714672,
  'max_markers': 1,
  'min_markers': 0,
  'result_markers': 0},
 {'description': 'Покупець надійшов',
  'avg_markers': 0.005801027834714672,
  'max_markers': 1,
  'min_markers': 0,
  'result_markers': 0},
 {'description': 'Черга покупців',
  'avg_markers': 0.009675905296616463,
  'max_markers': 2,
  'min_markers': 0,
  'result_markers': 0},
 {'description': 'Кількість покупців, що покинули мазагин',
  'avg_markers': 1397.326266821139,
  'max_markers': 4282,
  'min_markers': 0,
  'result_markers': 4282},
 {'description': 'Запас товару',
  'avg_markers': 400.48020675519115,
  'max_markers': 600,
  'min_markers': 0,
  'result_markers': 0},
 {'description': 'Продавець',
  'avg_markers': 0.8321456792323917,
  'max_markers': 1,
  'min_markers': 0,
  'result_markers': 1},
 {'description': 'Кількість обслугованих покупців',
  'avg_markers': 199.35193892404124,
  'max_markers': 600,
  'min_markers': 0,
  'result_markers':